# New Event Check
:auth: Nate Stevens  
:email: ntsteven@uw.edu  
:org: Pacific Northwest Seismic Network  
:license: GPLv3  
:purpose:  
This notebook demonstrates an integration of ObsPy, ObsPlus, and Pyrocko for running  
interactive, visual quality assessments on associations, events, and waveform data.  

In [1]:
# Install dependency - WARNING: DOES NOT WORK ON JUPYTER HUB
# ! pip install pyrocko[gui]

zsh:1: no matches found: pyrocko[gui]


In [2]:
from pathlib import Path
from obsplus import EventBank
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, Stream, Inventory
from obspy_compat2 import *

In [3]:
ROOT = Path.cwd()
print(ROOT)

/Users/nates/Code/GitHub/2025_ML_TSC/notebooks/Nate


In [4]:
# Add functionalities to ObsPy objects
obspy_compat.plant()
# Connect to the eventbank from catalog_management.ipynb
ebank = EventBank(base_path=ROOT/'catalog_files'/'EventBank')
# Confirmation checks that we connected to the right EventBank
df_eb = ebank.read_index()
if ebank.path_structure != '{year}/{month}/{day}/{hour}':
    raise ValueError('does not appear that we are pointing at our EventBank made in exercise 2')
else:
    display(df_eb.sort_values('magnitude'))
# Connect to Webservices from Northern California Earthquake Data Center
wclient = Client('NCEDC')

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,path
1082,2022-12-20 20:12:27.648,40.4553,-124.1846,-1098.0,-0.731,,15.0,NaN,smi:local/event/921,NaN,...,0.580,15.0,9.0,749.0,2025-05-12 20:24:14.640252416,,,NaT,,2022/12/20/20/921.xml
1077,2022-12-20 20:08:32.468,40.5779,-124.0120,1220.0,-0.442,,14.0,NaN,smi:local/event/916,NaN,...,0.566,14.0,10.0,655.0,2025-05-12 20:24:14.633922304,,,NaT,,2022/12/20/20/916.xml
885,2022-12-20 19:37:21.992,40.4496,-124.0100,-1984.0,-0.391,,14.0,NaN,smi:local/event/886,NaN,...,0.770,14.0,10.0,441.0,2025-05-12 20:23:49.982156544,,,NaT,,2022/12/20/19/886.xml
68,2022-12-20 10:56:09.447,40.3232,-124.3160,-668.0,-0.273,,11.0,NaN,smi:local/event/69,NaN,...,0.523,11.0,8.0,1004.0,2025-05-12 20:23:49.036283904,,,NaT,,2022/12/20/10/69.xml
1086,2022-12-20 20:16:43.431,40.5128,-124.1336,-1997.0,-0.200,,15.0,NaN,smi:local/event/925,NaN,...,1.111,15.0,12.0,540.0,2025-05-12 20:24:14.643932672,,,NaT,,2022/12/20/20/925.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,2022-12-20 22:06:47.583,39.8366,-123.2902,36555.0,4.027,,16.0,NaN,smi:local/event/1051,NaN,...,0.582,16.0,10.0,390.0,2025-05-12 20:23:50.149746432,,,NaT,,2022/12/20/22/1051.xml
510,2022-12-20 15:09:05.545,40.6037,-124.0766,24014.0,4.331,,86.0,NaN,smi:local/event/511,NaN,...,0.327,86.0,48.0,163.0,2025-05-12 20:23:49.587475456,,,NaT,,2022/12/20/15/511.xml
940,2022-12-20 22:06:34.276,40.5328,-124.3388,17929.0,4.508,,94.0,NaN,smi:local/event/1050,NaN,...,0.490,94.0,55.0,123.0,2025-05-12 20:23:50.149095424,,,NaT,,2022/12/20/22/1050.xml
21,2022-12-20 10:39:02.253,40.5212,-124.1818,27076.0,4.563,,93.0,NaN,smi:local/event/22,NaN,...,0.427,93.0,58.0,146.0,2025-05-12 20:23:48.973626368,,,NaT,,2022/12/20/10/22.xml


In [5]:
# Pick a number between 0 and 1161
myidx = 13
evid = df_eb.iloc[myidx].event_id
cat = ebank.get_events(event_id = evid)
display(cat)

1 Event(s) in Catalog:
2022-12-20T10:34:24.607000Z | +40.513, -124.421 | 6.09 None

We have to run some cross-checks to make sure picks and queried waveforms have the same NSLC codes

In [6]:
# Apply channel code updates to pick objects as they are able to match to waveform data
# A lot of this headache can be skipped if a full NSLC station index is saved... we don't have that from hypodd outputs.
st = Stream()
inv = Inventory()
client = Client('NCEDC')
front_pad = 10
back_pad = 30
for pick in cat[0].picks:
    tp = pick.time
    wfid = pick.waveform_id
    # Preferred heirarchy of band codes
    for _b in 'HE':
        # Preferred heirarchy of instrument codes
        for _i in 'HN':
            # Try to get waveform data
            try:
                _st = client.get_waveforms(wfid.network_code,
                                        wfid.station_code,
                                        '*',
                                        f'{_b}{_i}{wfid.channel_code}',
                                        tp - front_pad,
                                        tp + back_pad)
            except:
                continue

            if len(_st) > 0:
                pick.waveform_id.channel_code = _st[0].stats.channel
                pick.waveform_id.location_code = _st[0].stats.location
                display(pick.waveform_id.id)
                st += _st
                inv += client.get_stations(starttime=tp,
                                           network=pick.waveform_id.network_code,
                                           station=pick.waveform_id.station_code,
                                           location=pick.waveform_id.location_code,
                                           channel=pick.waveform_id.channel_code,
                                           level='channel')
                if pick.waveform_id.channel_code[-1] == 'N':
                    st += client.get_waveforms(wfid.network_code,
                                               wfid.station_code,
                                               '*',
                                               f'{_b}{_i}E',
                                               tp - front_pad,
                                               tp + back_pad)    
                
                continue

# Merge traces to deduplicate samples
st.merge()


'NP.1584B.10.HNZ'

'NC.KCT..HHZ'

'NP.1023.10.HNZ'

'NP.1586.10.HNZ'

'PB.B046..EHZ'

...

'NC.KBN.01.HNN'

'BK.DCMP.01.HHN'

'NC.GBL.01.HNN'

'BK.SCOT.01.HHN'

152 Trace(s) in Stream:

BK.BJES.00.HHE | 2022-12-20T10:34:31.150000Z - 2022-12-20T10:35:11.140000Z | 100.0 Hz, 4000 samples
...
(150 other traces)
...
PB.B935..EHZ | 2022-12-20T10:34:27.528393Z - 2022-12-20T10:35:07.518393Z | 100.0 Hz, 4000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [7]:
display(st)
display(inv)

152 Trace(s) in Stream:

BK.BJES.00.HHE | 2022-12-20T10:34:31.150000Z - 2022-12-20T10:35:11.140000Z | 100.0 Hz, 4000 samples
...
(150 other traces)
...
PB.B935..EHZ | 2022-12-20T10:34:27.528393Z - 2022-12-20T10:35:07.518393Z | 100.0 Hz, 4000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

Inventory created at 2025-05-13T15:29:43.275460Z
	Created by: ObsPy 1.4.2
		    https://www.obspy.org
	Sending institution: NCEDC,ObsPy 1.4.2 (NCEDC)
	Contains:
		Networks (97):
			BK (24x), CE (16x), NC (36x), NP (14x), PB (7x)
		Stations (97):
			BK.BJES (Bald Jesse, Humboldt County, CA) (2x)
			BK.BRIC (Briceland Vineyard, Redway, CA) (2x)
			BK.DCMP (DeCamp, Mendocino County, CA) (2x)
			BK.DMOR (Dinsmores, Humboldt County, CA) (2x)
			BK.HAYF (Tule Creek, Hayfork, CA) (2x)
			BK.KNEE (Kneeland/Ashfield Ridge) (2x)
			BK.MNDO (Woodlands Fire Station, Mendocino, CA)
			BK.PETL (Punta Gorda, Petrolia, CA)
			BK.PETY (Pettyjohn Place, Tehama County, CA)
			BK.PRDS (Paradise Ridge) (2x)
			BK.RBOW (Rainbow Ridge, Humboldt, CA)
			BK.RVIT (Round Valley Indian Tribe, Mendocino County, CA) (2x)
			BK.SCOT (Callahan, CA, USA) (2x)
			BK.WEAV (Weaverville, CA USA) (2x)
			CE.79036 (Covelo - CDF Fire Station)
			CE.79046 (Rockport A)
			CE.79298 (Fort Bragg - Hwy 1 & Ocean View)
			CE.79666 

In [ ]:
events, markers = to_pyrocko_events_and_picks(cat)
obspy_compat.snuffle(st, ntracks=len(st), inventory=inv, markers=markers)

Matplotlib Qt5Agg backend is not available. Snufflings drawing matplotlib figures may not work properly.
